In [1]:
## Necessary for running locally (Natascha's laptop)
## Make sure running in appropriate environement

import sys

sys.path.append('/Users/nataschaspahr/code/ai_synbio_data_processing')

In [6]:
import os
from binfo_utils import create_manifest

base_dir = '/Users/nataschaspahr/data/seq_data/test_short/'
received_dir = os.path.join(base_dir, 'received')

manifest = create_manifest(input_dir)
manifest

,sample_name,fwd_fastq,rvs_fastq
0,E2,/Users/nataschaspahr/data/seq_data/test_short/...,/Users/nataschaspahr/data/seq_data/test_short/...


### QC 1

In [3]:
from read_qc import run_fastqc

[run_fastqc(f) for f in manifest['fwd_fastq']]
[run_fastqc(f) for f in manifest['rvs_fastq']]

Running FastQC on /Users/nataschaspahr/data/seq_data/test_short/received/E2_S1_L001_R1_001.fastq.gz
application/gzip


Started analysis of E2_S1_L001_R1_001.fastq.gz
Approx 5% complete for E2_S1_L001_R1_001.fastq.gz
Approx 10% complete for E2_S1_L001_R1_001.fastq.gz
Approx 15% complete for E2_S1_L001_R1_001.fastq.gz
Approx 20% complete for E2_S1_L001_R1_001.fastq.gz
Approx 25% complete for E2_S1_L001_R1_001.fastq.gz
Approx 30% complete for E2_S1_L001_R1_001.fastq.gz
Approx 35% complete for E2_S1_L001_R1_001.fastq.gz
Approx 40% complete for E2_S1_L001_R1_001.fastq.gz
Approx 45% complete for E2_S1_L001_R1_001.fastq.gz
Approx 50% complete for E2_S1_L001_R1_001.fastq.gz
Approx 55% complete for E2_S1_L001_R1_001.fastq.gz
Approx 60% complete for E2_S1_L001_R1_001.fastq.gz
Approx 65% complete for E2_S1_L001_R1_001.fastq.gz
Approx 70% complete for E2_S1_L001_R1_001.fastq.gz
Approx 75% complete for E2_S1_L001_R1_001.fastq.gz
Approx 80% complete for E2_S1_L001_R1_001.fastq.gz
Approx 85% complete for E2_S1_L001_R1_001.fastq.gz
Approx 90% complete for E2_S1_L001_R1_001.fastq.gz
Approx 95% complete for E2_S1_L001_R

Analysis complete for E2_S1_L001_R1_001.fastq.gz


2025-09-15 17:38:46.943 java[45187:2486855] [JRSAppKitAWT markAppIsDaemon] failed. SetApplicationIsDaemon returned -50


Running FastQC on /Users/nataschaspahr/data/seq_data/test_short/received/E2_S1_L001_R2_001.fastq.gz
application/gzip


Started analysis of E2_S1_L001_R2_001.fastq.gz
Approx 5% complete for E2_S1_L001_R2_001.fastq.gz
Approx 10% complete for E2_S1_L001_R2_001.fastq.gz
Approx 15% complete for E2_S1_L001_R2_001.fastq.gz
Approx 20% complete for E2_S1_L001_R2_001.fastq.gz
Approx 25% complete for E2_S1_L001_R2_001.fastq.gz
Approx 30% complete for E2_S1_L001_R2_001.fastq.gz
Approx 35% complete for E2_S1_L001_R2_001.fastq.gz
Approx 40% complete for E2_S1_L001_R2_001.fastq.gz
Approx 45% complete for E2_S1_L001_R2_001.fastq.gz
Approx 50% complete for E2_S1_L001_R2_001.fastq.gz
Approx 55% complete for E2_S1_L001_R2_001.fastq.gz
Approx 60% complete for E2_S1_L001_R2_001.fastq.gz
Approx 65% complete for E2_S1_L001_R2_001.fastq.gz
Approx 70% complete for E2_S1_L001_R2_001.fastq.gz
Approx 75% complete for E2_S1_L001_R2_001.fastq.gz
Approx 80% complete for E2_S1_L001_R2_001.fastq.gz
Approx 85% complete for E2_S1_L001_R2_001.fastq.gz
Approx 90% complete for E2_S1_L001_R2_001.fastq.gz
Approx 95% complete for E2_S1_L001_R

Analysis complete for E2_S1_L001_R2_001.fastq.gz


2025-09-15 17:38:55.685 java[45218:2487314] [JRSAppKitAWT markAppIsDaemon] failed. SetApplicationIsDaemon returned -50


[None]

### Trimming

In [7]:
import subprocess
from binfo_utils import create_manifest

trimmed_dir = os.path.join(base_dir, 'trimmed')
os.makedirs(trimmed_dir, exist_ok=True)

manifest = create_manifest(received_dir)
for index, row in manifest.iterrows():

    read1 = str(os.path.basename(row['fwd_fastq'])).replace('.fastq.gz', '_trimmed.fastq.gz') 
    read2 = str(os.path.basename(row['rvs_fastq'])).replace('.fastq.gz', '_trimmed.fastq.gz')
    
    cutadapt_cmd = [
        'cutadapt',
        '-j', '10',
        '-q', '15',
        '-a', 'AGATCGGAAGAGCACACGTCTGAACTCCAGTCA',
        '-A', 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT',
        '--poly-a',
        '--minimum-length',  '20',
        '-o', os.path.join(trimmed_dir, read1),
        '-p', os.path.join(trimmed_dir, read2),
        row['fwd_fastq'],
        row['rvs_fastq'],
    ]

    subprocess.run(cutadapt_cmd, check=True, cwd=trimmed_dir)

This is cutadapt 5.1 with Python 3.12.2
Command line parameters: -j 10 -q 15 -a AGATCGGAAGAGCACACGTCTGAACTCCAGTCA -A AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGT --poly-a --minimum-length 20 -o /Users/nataschaspahr/data/seq_data/test_short/trimmed/E2_S1_L001_R1_001_trimmed.fastq.gz -p /Users/nataschaspahr/data/seq_data/test_short/trimmed/E2_S1_L001_R2_001_trimmed.fastq.gz /Users/nataschaspahr/data/seq_data/test_short/received/E2_S1_L001_R1_001.fastq.gz /Users/nataschaspahr/data/seq_data/test_short/received/E2_S1_L001_R2_001.fastq.gz
Processing paired-end reads on 10 cores ...

=== Summary ===

Total read pairs processed:          1,117,271
  Read 1 with adapter:               1,022,526 (91.5%)
  Read 2 with adapter:               1,011,253 (90.5%)

== Read fate breakdown ==
Pairs that were too short:               1,022 (0.1%)
Pairs written (passing filters):     1,116,249 (99.9%)

Total basepairs processed:   560,870,042 bp
  Read 1:   280,435,021 bp
  Read 2:   280,435,021 bp
Quality-trimmed:  

### QC 2

In [8]:
from read_qc import run_fastqc
from binfo_utils import create_manifest

manifest = create_manifest(trimmed_dir)

[run_fastqc(f) for f in manifest['fwd_fastq']]
[run_fastqc(f) for f in manifest['rvs_fastq']]

Running FastQC on /Users/nataschaspahr/data/seq_data/test_short/trimmed/E2_S1_L001_R1_001_trimmed.fastq.gz
application/gzip


Started analysis of E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 5% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 10% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 15% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 20% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 25% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 30% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 35% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 40% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 45% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 50% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 55% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 60% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 65% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 70% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 75% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Approx 80% complete for E2_S1_L001_R1_001_trimmed.fastq.gz
Ap

Analysis complete for E2_S1_L001_R1_001_trimmed.fastq.gz


2025-09-15 18:03:35.722 java[46169:2507864] [JRSAppKitAWT markAppIsDaemon] failed. SetApplicationIsDaemon returned -50


Running FastQC on /Users/nataschaspahr/data/seq_data/test_short/trimmed/E2_S1_L001_R2_001_trimmed.fastq.gz
application/gzip


Started analysis of E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 5% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 10% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 15% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 20% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 25% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 30% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 35% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 40% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 45% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 50% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 55% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 60% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 65% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 70% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 75% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Approx 80% complete for E2_S1_L001_R2_001_trimmed.fastq.gz
Ap

Analysis complete for E2_S1_L001_R2_001_trimmed.fastq.gz


2025-09-15 18:03:41.727 java[46173:2508030] [JRSAppKitAWT markAppIsDaemon] failed. SetApplicationIsDaemon returned -50


[None]

In [14]:
%load_ext autoreload
%autoreload 2

In [20]:
from mapping import map_reads

fwd_reads = manifest['fwd_fastq'][0]
rvs_reads = manifest['rvs_fastq'][0]
reference = '/Users/nataschaspahr/Reference_DBs/genomes/ACN2821_CDM.gbk'
keep_index = False
index_dir = os.path.join(base_dir, 'index')
mapped_dir = os.path.join(base_dir, 'mapped')
os.makedirs(index_dir, exist_ok=True)
os.makedirs(mapped_dir, exist_ok=True)

map_reads(fwd_reads, rvs_reads, reference, mapped_dir, keep_index=keep_index, index_dir=index_dir)

# import os
# os.path.splitext(reference)

Building bowtie2 index in /var/folders/zk/qjhjrlq57411m2lc03t3f36r0000gp/T/tmp95sqtarj from reference /Users/nataschaspahr/Reference_DBs/genomes/ACN2821_CDM.gbk
Settings:
  Output files: "/var/folders/zk/qjhjrlq57411m2lc03t3f36r0000gp/T/tmp95sqtarj/reference.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /var/folders/zk/qjhjrlq57411m2lc03t3f36r0000gp/T/tmp95sqtarj/reference.fasta
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to j

/opt/anaconda3/envs/bre_env/lib/python3.12/site-packages/Bio/GenBank/Scanner.py:1530: BiopythonParserWarning: Attempting to parse malformed locus line:
'LOCUS       NC_005966     3599271 bp    DNA     circular UNA 29-JUL-2025\n'
Found locus 'NC_005966' size '3599271' residue_type 'DNA'
Some fields may be wrong.
  warnings.warn(
Building a SMALL index


Splitting and merging
  Splitting and merging time: 00:00:00
Split 1, merged 8; iterating...
Splitting and merging
  Splitting and merging time: 00:00:00
Split 2, merged 0; iterating...
Splitting and merging
  Splitting and merging time: 00:00:00
Avg bucket size: 514181 (target: 674862)
Converting suffix-array elements to index image
Allocating ftab, absorbFtab
Entering Ebwt loop
Getting block 1 of 7
  Reserving size (674863) for bucket 1
  Calculating Z arrays for bucket 1
  Entering block accumulator loop for bucket 1:
  bucket 1: 10%
  bucket 1: 20%
  bucket 1: 30%
  bucket 1: 40%
  bucket 1: 50%
  bucket 1: 60%
  bucket 1: 70%
  bucket 1: 80%
  bucket 1: 90%
  bucket 1: 100%
  Sorting block of length 342142 for bucket 1
  (Using difference cover)
  Sorting block time: 00:00:00
Returning block of 342143 for bucket 1
Getting block 2 of 7
  Reserving size (674863) for bucket 2
  Calculating Z arrays for bucket 2
  Entering block accumulator loop for bucket 2:
  bucket 2: 10%
  bucket 

Renaming /var/folders/zk/qjhjrlq57411m2lc03t3f36r0000gp/T/tmp95sqtarj/reference.3.bt2.tmp to /var/folders/zk/qjhjrlq57411m2lc03t3f36r0000gp/T/tmp95sqtarj/reference.3.bt2
Renaming /var/folders/zk/qjhjrlq57411m2lc03t3f36r0000gp/T/tmp95sqtarj/reference.4.bt2.tmp to /var/folders/zk/qjhjrlq57411m2lc03t3f36r0000gp/T/tmp95sqtarj/reference.4.bt2
Renaming /var/folders/zk/qjhjrlq57411m2lc03t3f36r0000gp/T/tmp95sqtarj/reference.1.bt2.tmp to /var/folders/zk/qjhjrlq57411m2lc03t3f36r0000gp/T/tmp95sqtarj/reference.1.bt2
Renaming /var/folders/zk/qjhjrlq57411m2lc03t3f36r0000gp/T/tmp95sqtarj/reference.2.bt2.tmp to /var/folders/zk/qjhjrlq57411m2lc03t3f36r0000gp/T/tmp95sqtarj/reference.2.bt2
Renaming /var/folders/zk/qjhjrlq57411m2lc03t3f36r0000gp/T/tmp95sqtarj/reference.rev.1.bt2.tmp to /var/folders/zk/qjhjrlq57411m2lc03t3f36r0000gp/T/tmp95sqtarj/reference.rev.1.bt2
Renaming /var/folders/zk/qjhjrlq57411m2lc03t3f36r0000gp/T/tmp95sqtarj/reference.rev.2.bt2.tmp to /var/folders/zk/qjhjrlq57411m2lc03t3f36r0000g

Sorting and indexing BAM file from /Users/nataschaspahr/data/seq_data/test_short/mapped/output.sam


[bam_sort_core] merging from 1 files and 1 in-memory blocks...


'/Users/nataschaspahr/data/seq_data/test_short/mapped/output.sorted.bam'